In [1]:
#IMPORTS

#File IO
import os
import glob

#Data manipulation
import numpy as np

#Pytorch
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

#Misc
from tqdm import tqdm


In [2]:
#GET TARGET DATA

def get_features(file):
    with np.load(file) as data:
        bands = data['rad']
        features = np.stack([
            data['l2_cloud_mask'],
            data['l2_cloud_phase'],
            data['l2_cod'] * 0.001,
            data['l2_cps'] * 0.001
        ])
    return bands, features

In [3]:
max_images = 15000

#CREATE DATASET

file_list = glob.glob('../ABI_Data/*.npz')

X = []
y = []

for i, file in tqdm(enumerate(file_list)):
    features, target = get_features(file)
    X.append(features)
    y.append(target)
    if i == max_images-1: break

X = np.stack(X)  # shape: (Samples, 128, 128, 16)
X = torch.from_numpy(X).float()
X = X.permute(0, 3, 1, 2)  # shape: (Samples, 16, 128, 128)


y = np.stack(y)
y = torch.from_numpy(y).float() # shape: (Samples, 4, 128, 128)

dataset = TensorDataset(X, y)
torch.save(dataset, "dataset.pt")

123it [00:35,  3.48it/s]


KeyboardInterrupt: 